In [39]:
import pygame
import numpy as np
import tensorflow as tf
import time
import os
from Core_Game_Parts import *
from keras import layers, models


In [40]:
def build_dueling_dqn(input_shape=(4,), action_size=3):
    from keras import layers, models, backend as K
    import tensorflow as tf

    inputs = layers.Input(shape=input_shape)
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.Dense(128, activation='relu')(x)

    # Dueling streams
    value = layers.Dense(64, activation='relu')(x)
    value = layers.Dense(1, activation='linear')(value)

    advantage = layers.Dense(64, activation='relu')(x)
    advantage = layers.Dense(action_size, activation='linear')(advantage)

    # Correct combination using Lambda layers to maintain graph compatibility
    def combine_dueling(inputs):
        value, advantage = inputs
        return value + (advantage - tf.reduce_mean(advantage, axis=1, keepdims=True))

    q_values = layers.Lambda(combine_dueling)([value, advantage])

    model = models.Model(inputs=inputs, outputs=q_values)
    return model
    
def build_dueling_dqn(input_shape=(5,), action_size=4):
    from keras import layers, models, backend as K
    import tensorflow as tf

    inputs = layers.Input(shape=input_shape)
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.Dense(128, activation='relu')(x)

    # Dueling streams
    value = layers.Dense(64, activation='relu')(x)
    value = layers.Dense(1, activation='linear')(value)

    advantage = layers.Dense(64, activation='relu')(x)
    advantage = layers.Dense(action_size, activation='linear')(advantage)

    # Correct combination using Lambda layers to maintain graph compatibility
    def combine_dueling(inputs):
        value, advantage = inputs
        return value + (advantage - tf.reduce_mean(advantage, axis=1, keepdims=True))

    q_values = layers.Lambda(combine_dueling)([value, advantage])

    model = models.Model(inputs=inputs, outputs=q_values)
    return model


In [41]:
def run_d3qn_agent(weights_path, render=True):
    os.environ["SDL_VIDEODRIVER"] = "windows"
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    
    
    START_LINE_RECT = pygame.Rect(860, 400, 80, 60) 
    pygame.display.set_caption("Trained D3QN Agent")
    font = pygame.font.SysFont(None, 24)
    crash_font = pygame.font.SysFont(None, 72)

    track_surface = pygame.image.load(TRACK_IMAGE_PATH).convert()

    # Preserved model shape from Code 1 (5 inputs, 4 actions)
    model = build_dueling_dqn(input_shape=(5,), action_size=4)
    model.load_weights(weights_path)

    car = Car(CAR_IMAGE_PATH, 900, 426, angle=-45)
    car.speed = 0
    lap_started = False
    lap_start_time = None
    last_lap_time = None
    on_start_line = False
    can_finish_lap = False
    MIN_LAP_TIME = 5.0 
    # --- COUNTERS ADDED ---
    action_counts = {0: 0, 1: 0, 2: 0, 3: 0} 
    step_counter = 0

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        distances, rays = ray_casting(car, track_surface)
        left, front, right = distances
        left /= 200.0
        front /= 200.0
        right /= 200.0
        curvature = abs(left - right) / max(left + right, 1e-6)

        state = np.array([left, front, right, car.speed, curvature], dtype=np.float32)
        state = np.expand_dims(state, axis=0)

        q = model.predict(state, verbose=0)[0]
        action = np.argmax(q)

        # Update stats
        action_counts[action] += 1
        step_counter += 1

        # MATCH TRAINING PHYSICS (Preserved from Code 1)
        if action == 0:
            car.angle += 5
            car.speed = max(car.speed - 0.12, 2.0)
        elif action == 2:
            car.angle -= 5
            car.speed = max(car.speed - 0.12, 2.0)
        elif action == 1:
            car.speed = min(car.speed + 0.15, 10.0)
        elif action == 3:
            car.speed = max(car.speed - 0.30, 1.8)

        car.move()

        car_rect = car.get_rect()
        touching_line = car_rect.colliderect(START_LINE_RECT)

        # ---------------------------
        # EDGE DETECTION
        # ---------------------------
        if touching_line and not on_start_line:
            on_start_line = True

            # START LAP
            if not lap_started:
                lap_started = True
                lap_start_time = time.time()
                can_finish_lap = False
                print("Lap started!")

            # FINISH LAP (only if allowed)
            elif lap_started and can_finish_lap:
                lap_time = time.time() - lap_start_time

                if lap_time >= MIN_LAP_TIME:
                    print("\nLAP COMPLETED!")
                    print(f"Lap Time: {lap_time:.2f} seconds")
                    print(f"Steps: {step_counter}")
                    print(f"Action counts: {action_counts}\n")

                    last_lap_time = lap_time
                    lap_started = False
                    lap_start_time = None
                    can_finish_lap = False

        # ---------------------------
        # LEAVING START LINE
        # ---------------------------
        if not touching_line and on_start_line:
            on_start_line = False
            if lap_started:
                can_finish_lap = True


        # Crash check with Text Display (Integrated features)
        crashed = False
        try:
            if track_surface.get_at((int(car.x), int(car.y)))[:3] == DRAW_COLOR:
                crashed = True
        except:
            crashed = True

        if crashed:
            # Display crash message
            crash_text = crash_font.render("CRASHED! Resetting...", True, (255, 0, 0))
            text_rect = crash_text.get_rect(center=(SCREEN_WIDTH/2, SCREEN_HEIGHT/2))
            screen.blit(crash_text, text_rect)
            pygame.display.update()
            print("\nCRASH!")
            if lap_started:
                print(f"Time before crash: {time.time() - lap_start_time:.2f}s")
            print(f"Steps: {step_counter}")
            print(f"Action counts: {action_counts}\n")

            # Wait 2 seconds to see the crash
            pygame.time.wait(2000)
            
            # Reset Car and Counters
            car = Car(CAR_IMAGE_PATH, 900, 426, angle=-45)
            car.speed = 0
            action_counts = {0: 0, 1: 0, 2: 0, 3: 0}
            step_counter = 0
            lap_started = False
            lap_start_time = None
            last_lap_time = None
            on_start_line = False
            can_finish_lap = False



        if render:
            screen.blit(track_surface, (0, 0))
            car.draw(screen)
            for p in rays:
                pygame.draw.line(screen, (0, 255, 0), (car.x, car.y), p, 1)
            
            # --- HUD / STATS DISPLAY ---
            # Line 1: Speed and Steps
            info_text = f"Speed: {car.speed:.2f} | Steps: {step_counter}"
            text_surface = font.render(info_text, True, (255, 255, 255))
            screen.blit(text_surface, (20, 20))
            
            # Line 2: Action Counts
            # Mapping based on logic: 0:Left, 1:Accel, 2:Right, 3:Brake
            stats_text = f"Actions -> L(0):{action_counts[0]}  Acc(1):{action_counts[1]}  R(2):{action_counts[2]}  Brk(3):{action_counts[3]}"
            stats_surface = font.render(stats_text, True, (255, 255, 255))
            screen.blit(stats_surface, (20, 50))
            pygame.draw.rect(screen, (255, 255, 0), START_LINE_RECT, 2)

            if lap_started and lap_start_time is not None:
                elapsed = time.time() - lap_start_time
                timer_text = f"Lap Time: {elapsed:.2f}s"
            elif last_lap_time is not None:
                timer_text = f"Last Lap: {last_lap_time:.2f}s"
            else:
                timer_text = "Lap Time: --"

            timer_surface = font.render(timer_text, True, (255, 255, 0))
            screen.blit(timer_surface, (20, 80))


            pygame.display.flip()
            clock.tick(60)
    

    pygame.quit()

In [45]:
WEIGHTS_FILENAME = "best_d3qn_16checkpoints.weights.h5"   # or whatever you named it d3qn.weights.h5

if not pygame.display.get_init():
    run_d3qn_agent(WEIGHTS_FILENAME, render=True)
else:
    print("Pygame window may already be open. Restart the kernel and try again.")  # 14

Lap started!

LAP COMPLETED!
Lap Time: 37.39 seconds
Steps: 676
Action counts: {0: 137, 1: 308, 2: 221, 3: 10}

